# Implementing UMAP from scratch


In [ ]:
import random

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml

import scipy
import scipy.sparse
from scipy.optimize import curve_fit
import numba

from pynndescent import NNDescent

sns.set_theme()
random.seed(2)

## Load the digits dataset

In [ ]:
# Load the digits data
mnist = fetch_openml('mnist_784', parser='auto')

# Normalize digits data
mnist_data = mnist.data/255
mnist_label = mnist.target.astype(object)

# Downsample
mnist_label = mnist_label[(mnist_label.isin(['0', '1', '2', '3', '4']))]

mnist_data = mnist_data.loc[mnist_label.index, :].reset_index(drop=True)

mnist_label = mnist_label.reset_index(drop=True)

In [ ]:
mnist_data.shape

In [ ]:
mnist_label.unique()

In [ ]:
# Initialize a dictionary to store the first instance of each number 0-4
first_instance = {}

# Loop through the dataset to find the first instance for numbers 0-4
for i in range(len(mnist_label)):
    
    label = mnist_label[i]
    
    if label not in first_instance:
        first_instance[label] = i

# Plotting
fig, axes = plt.subplots(1, 5, figsize=(10, 2))

for label, index in first_instance.items():
    ax = axes[int(label)]
    ax.imshow(mnist_data.loc[index].values.reshape(28, 28), cmap='gray')
    ax.set_title(f'Label: {label}')
    ax.axis('off')

plt.show()

## Building a graph


### Find nearest neighbours and distances for each point



In [ ]:
N_NEIGHBORS = 20
index = NNDescent(mnist_data, n_neighbors=N_NEIGHBORS)
neighbors, distances = index.query(mnist_data, k=N_NEIGHBORS)

In [ ]:
neighbors.shape

In [ ]:
distances.shape

In [ ]:
print(neighbors[:20,:5])

In [ ]:
print(distances[:20,:5])

In [ ]:
neighbors = neighbors[:, 1:]
distances = distances[:, 1:]

In [ ]:
plt.figure()
plt.plot(np.arange(distances.shape[1], dtype=np.int32)+1, distances[0,:])
plt.xlabel('Nth nearest neighbor')
plt.xticks(np.arange(distances.shape[1], dtype=np.int32)+1)
plt.ylabel('Distance')
plt.title('Neighbor distance from "data point 0"')

### Calculate rho for each data point

$$
\rho_i = \min_{j \in S} d(i, j)
$$



In [ ]:
rhos = distances[:, 0]
print(rhos[:5])

### Calculate sigmas


$$
sim_{ij} = \exp \left(- \frac{\max(0, d(x_i, x_{ij}) - \rho_i)}{\sigma_i} \right)
$$


$$
\sigma_i = \text{the value such that } \sum_{j=1}^{k} \exp \left(- \frac{\max(0, d(x_i, x_{ij}) - \rho_i)}{\sigma_i} \right) = \log_2(k)
$$


In [ ]:
# Set parameter values for binary search
BINARY_ITER = 64
GAMMA_TOLERANCE = 1e-5

# Log target is same everywhere
log_target = np.log2(N_NEIGHBORS)

In [ ]:
# Example - binary search for a single row
curr_dist = distances[0,:]

# Initialize array of zeros
zero_array = np.zeros(curr_dist.shape[0])

# Set up binary search for sigma
sigma_high = np.inf
sigma_low = 0
sigma_mid = 1

# Perform N itterations
for iter_i in range(BINARY_ITER):
    
    # Substract distances and rho 
    dist_diff = curr_dist - rhos[0]

    # Divide by sigma, negate, perform exponent, sum
    sigma_sum = np.sum(np.exp(-dist_diff/sigma_mid))
    
    # Break the loop if sum is equal to the target
    if np.abs(sigma_sum-log_target) < GAMMA_TOLERANCE:
        break
    
    # If obtained sum is greater than the target then
    # make sigma smaller
    if sigma_sum > log_target:
        
        sigma_high = sigma_mid
        sigma_mid = (sigma_high + sigma_low)/2
    
    # If obtained sum is lower than the target then
    # make sigma higher
    else:
        sigma_low = sigma_mid
        if sigma_high == np.inf:
            sigma_mid = sigma_mid*2
        else:
            sigma_mid = (sigma_high + sigma_low)/2
            
print('Obtained sigma sum is : {}'.format(sigma_sum))
print('Target sigma sum is : {}'.format(log_target))
print('Value of sigma is : {}'.format(sigma_mid))

In [ ]:
sims_1 = np.exp(-((curr_dist - rhos[0]) / (sigma_mid)))

plt.figure()
plt.plot(np.arange(distances.shape[1], dtype=np.int32)+1, sims_1)
plt.xlabel('Nth nearest neighbor')
plt.xticks(np.arange(distances.shape[1], dtype=np.int32)+1)
plt.ylabel('Similarity')
plt.title('Neighbor similarity to "data point 0"')


In [ ]:
# Wrap binary search into function
def binary_search_for_sigma(curr_dist, curr_rho, log_target, binary_iter, smooth_k_tolerance):
    
    # Set up binary search for sigma
    sigma_high = np.inf
    sigma_low = 0
    sigma_mid = 1

    # Perform N itterations
    for _ in range(binary_iter):

        # Substract distances and rho 
        dist_diff = curr_dist - curr_rho

        # Divide by sigma, negate, perform exponent, sum
        sigma_sum = np.sum(np.exp(-dist_diff/sigma_mid))

        # Break the loop if sum is equal to the target
        if np.abs(sigma_sum-log_target) < smooth_k_tolerance:
            return sigma_mid

        # If obtained sum is greater than the target then
        # make sigma smaller
        if sigma_sum > log_target:

            sigma_high = sigma_mid
            sigma_mid = (sigma_high + sigma_low)/2

        # If obtained sum is lower than the target then
        # make sigma higher
        else:
            sigma_low = sigma_mid
            if sigma_high == np.inf:
                sigma_mid = sigma_mid*2
            else:
                sigma_mid = (sigma_high + sigma_low)/2

    return sigma_mid

In [ ]:
# Calculate sigma value for each sample
sigma_values = np.zeros(distances.shape[0])

for sample_i in range(distances.shape[0]):
    
    sigma_values[sample_i] = binary_search_for_sigma(
        curr_dist=distances[sample_i,:], 
        curr_rho=rhos[sample_i], 
        log_target=log_target,
        binary_iter=BINARY_ITER,
        smooth_k_tolerance=GAMMA_TOLERANCE
    )

### Calculate similarities between points

Similarities to neighbours are calculated as:

$$
sim_{ij} = \exp \left(- \frac{\max(0, d(x_i, x_{ij}) - \rho_i)}{\sigma_i} \right)
$$

This similarities are "simetrized" after calculation.

Formula for all the samples:
$$
S = P + P^T - P * P^T
$$

Example formula for two samples:
$$
S = P_(ij) + P_(ji) - P_(ij) * P_(ji)
$$

In [ ]:
# Get number of samples and choosen number of neighbours
n_samples = neighbors.shape[0]
n_neighbors = neighbors.shape[1]

# Initialize row, col and val arrays, this will later
# be used to construct sparse matrix in order to symetrize
# smoothed distance matrix.
row_idx = np.zeros(neighbors.size, dtype=np.int32) # will be used to store index of the 
col_idx = np.zeros(neighbors.size, dtype=np.int32) # will be used to store index of the neighbours
mtx_vals = np.zeros(neighbors.size, dtype=np.float32) # will be used to store similarity between sample and neighboor

# Loop goint through samples
for i in range(n_samples):
    # Inner loop going through the neighbours
    for j in range(n_neighbors):
        
        # Calculate similarities in high dimensional space
        val = np.exp(-((distances[i, j] - rhos[i]) / (sigma_values[i])))
        
        # store sample indexes e.g. [1,1,1,1,2,2,2,2]
        row_idx[i * n_neighbors + j] = i
        # store neighbour indexes e.g. [2,3,4,5,2,3,4,5]
        col_idx[i * n_neighbors + j] = neighbors[i, j]
        # store similarities e.g [0.1, 0.3, 0.2, 0.5, 0.8,0.12, 0.19, 0.97]
        mtx_vals[i * n_neighbors + j] = val
    


In [ ]:
# Create sparse matrix of smoothed distances
# Distances in this matrix are not yet symetric
non_sym_sim_mtx = scipy.sparse.coo_matrix(
    (mtx_vals, (row_idx, col_idx)), shape=(neighbors.shape[0], neighbors.shape[0])
)
non_sym_sim_mtx.eliminate_zeros()

In [ ]:
non_sym_sim_mtx.shape

In [ ]:
# Transpose smooth distance matrix
non_sym_sim_mtx_t = non_sym_sim_mtx.transpose()

# Multiply smooth distance matrix with its transponse element-wise
prod_matrix = non_sym_sim_mtx.multiply(non_sym_sim_mtx_t)

# Create symetrical smooth distance matrix by using fuzzy set union operation
sim_mtx = non_sym_sim_mtx + non_sym_sim_mtx_t - prod_matrix

# Eliminate all zeroes from the result
sim_mtx.eliminate_zeros()

In [ ]:
# Transform sparse matrix to coordinate format
sim_mtx = sim_mtx.tocoo()

In [ ]:
# Return to the old format
sim_mtx_row_ids = sim_mtx.row
sim_mtx_col_ids = sim_mtx.col
sim_mtx_vals = sim_mtx.data

In [ ]:
# Show how indexes and coordinates look like
# Also show identity of each index (for both samples and neighbours)
test = pd.DataFrame(
    np.column_stack([sim_mtx_row_ids, sim_mtx_col_ids, sim_mtx_vals]),
    columns = ['row', 'col', 'similarities']
)

ri_labels = mnist_label.reset_index(drop=True)
row_labels = ri_labels[sim_mtx_row_ids]
col_labels = ri_labels[sim_mtx_col_ids]

test.head()

## Embedding the graph into 2D space

### Calculate parameters a and b

In low dimensional space, similarity scores are calculated based on following formula:

$$
Sim(x, y) = \frac{1}{1 + a \cdot d^{(2 \cdot b)}}
$$


Parameters a and b are optimized so Sim(x, y) approximates following values:

$$
Target vals(x, y) =
\begin{cases}
1, & \text{if } d \le \text{min-dist} \\
\exp\left(-\left(d - \text{min-dist}\right)\right), & \text{otherwise}
\end{cases}
$$

In [ ]:
# Set up hyper parameters
MIN_DIST = 0.1
N_COMPONENTS = 2
N_EPOCHS = 2000
N_NEGATIVE = 8
LEARNING_RATE = 4

In [ ]:
def optim_curve(x, a, b):
    return 1.0 / (1.0 + a * x ** (2 * b))
    
# 300 simulated points between 0 and 3
# this axis simulates distances between
# points in low dimensional space
sim_dist = np.linspace(0, 3, 300)
# placehodler for target valyes 
target_vals = np.zeros(sim_dist.shape)
# set distances smaller than min_dist to 1
target_vals[sim_dist < MIN_DIST] = 1.0
# set distances greater than min_dist
target_vals[sim_dist >= MIN_DIST] = np.exp(-(sim_dist[sim_dist >= MIN_DIST] - MIN_DIST)
)
# optimize a and b parameters of function defined by "optim_curve"
# in order to produce target_vals when sim_dist is inputed
params, _ = curve_fit(optim_curve, sim_dist, target_vals)

a,b = params[0], params[1]

In [ ]:
print('Value of a is {}'.format(a))
print('Value of b is {}'.format(b))

In [ ]:
plt.plot(sim_dist, target_vals)
plt.xlabel('Distance')
plt.ylabel('Similarity')
plt.title('Target similarity')

In [ ]:
sim_dist = np.linspace(0, 3, 300)
similarities = optim_curve(sim_dist, a, b)
plt.plot(sim_dist, similarities)
plt.xlabel('Distance')
plt.ylabel('Similarity')
plt.title('Fitted curve')

### Low dimensional embedding optimization


In [ ]:
# Initialize embedding with random values
embedding = np.random.uniform(
    low=-3.0, high=3.0, size=(sim_mtx.shape[0], N_COMPONENTS)
)

In [ ]:
def euclidean_optim_one_epoch(sim_mtx_row_ids, sim_mtx_col_ids, a, b, embedding, weights, learning_rate, n_negative):
    
    # Iterate through pairs of samples and neighboirs 
    # i represents sample index
    # j represents neighbour index
    for i,j in zip(sim_mtx_row_ids, sim_mtx_col_ids):
        
        # Get embeddings for sample and index
        p1 = embedding[i, :]
        p2 = embedding[j, :]
        
        # Find euclidean distance in low dimensional space
        dist_squared = np.sum(np.square(p1-p2))
        
        # Do full gradient calculation for neighbours if distance is greater than zero
        if dist_squared > 0:
            curr_grad = (
                weights[i]*2*a*b*dist_squared**(b-1) / (a*dist_squared**b + 1.) 
                - (1-weights[i])*(2*b / ((0.001 + dist_squared)*(1+a*dist_squared**b)))
            )
        else:
            curr_grad = 0.
        
        curr_grad = curr_grad*(p1-p2)
        
        # Clip the gradients
        curr_grad = np.clip(curr_grad, -4., 4.)
        
        # Apply gradient to both, sample an neighbour
        embedding[i, :] -= curr_grad*learning_rate
        embedding[j, :] += curr_grad*learning_rate
        
        # Sample N random points and use them as "negative sample"
        # Negative sample is the sample that is not connected to the current sample
        for _ in range(n_negative):
            
            # Get randomly sample index
            neg_j = np.random.randint(embedding.shape[0])
            
            # Get embedding of negative sample
            p_neg = embedding[neg_j, : ]
            
            # Find euclidean distance between current sample and 
            # negative sample
            dist_squared = np.sum(np.square(p1-p_neg))
            
            # Calculate gradient (formula is shorter because we assume weights[i]==0)
            # for thus case
            if dist_squared > 0.:
                curr_grad = -2.*b / ((0.001 + dist_squared)*(1+a*dist_squared**b))
            else:
                curr_grad = 0.

            curr_grad = curr_grad*(p1-p_neg)
            
            # Clip the gradient
            curr_grad = np.clip(curr_grad, -4., 4.)
            
            # Apply gradient to the both, sample and the neighbour
            embedding[i, :] -=  curr_grad*learning_rate
            embedding[neg_j, :] +=  curr_grad*learning_rate


# Optimize the function by using numba
optimize_fn = numba.njit(
    euclidean_optim_one_epoch, fastmath=True, parallel=False
)

In [ ]:
euclidean_optim_one_epoch(
    sim_mtx_row_ids=sim_mtx_row_ids, 
    sim_mtx_col_ids=sim_mtx_col_ids, 
    a=a, 
    b=b, 
    embedding=embedding, 
    weights=sim_mtx_vals,
    learning_rate=LEARNING_RATE,
    n_negative=N_NEGATIVE
)

In [ ]:
optimize_fn(
    sim_mtx_row_ids=sim_mtx_row_ids, 
    sim_mtx_col_ids=sim_mtx_col_ids, 
    a=a, 
    b=b, 
    embedding=embedding, 
    weights=sim_mtx_vals,
    learning_rate=LEARNING_RATE,
    n_negative=N_NEGATIVE
)

In [ ]:
# Optimize in a loop
for epoch_i in range(200):
    optimize_fn(
        sim_mtx_row_ids=sim_mtx_row_ids, 
        sim_mtx_col_ids=sim_mtx_col_ids, 
        a=a, 
        b=b, 
        embedding=embedding, 
        weights=sim_mtx_vals,
        learning_rate=LEARNING_RATE,
        n_negative=N_NEGATIVE
    )
    
    if epoch_i%20 == 0:
        print(epoch_i)
        test_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'])
        test_df['label'] = mnist_label.values
        sns.scatterplot(test_df,x='UMAP1', y='UMAP2', hue='label')
        plt.show()

### Alternative function implementation with numba parallel

In [ ]:
def euclidean_optim_one_epoch(sim_mtx_row_ids, sim_mtx_col_ids, a, b, embedding, weights, learning_rate, n_negative):

    # In order to uze numpa parallel, loop must be performed with numba.prange
    for iter_i in numba.prange(sim_mtx_row_ids.shape[0]):
        
        # Extract index for sample and neighbour
        i = sim_mtx_row_ids[iter_i]
        j = sim_mtx_col_ids[iter_i]

        # Get embeddings for sample and neighbour
        p1 = embedding[i, :]
        p2 = embedding[j, :]
        
        # Find euclidean distance in low dimensional space
        dist_squared = np.sum(np.square(p1-p2))
        
        # Do full gradient calculation for neighbours if distance is greater than zero
        if dist_squared > 0:
            curr_grad = (
                weights[i]*2*a*b*dist_squared**(b-1) / (a*dist_squared**b + 1.) 
                - (1-weights[i])*(2*b / ((0.001 + dist_squared)*(1+a*dist_squared**b)))
            )
        else:
            curr_grad = 0.
        
        curr_grad = curr_grad*(p1-p2)
        
        # Clip the gradients
        curr_grad = np.clip(curr_grad, -4., 4.)
        
        # Apply gradient to both, sample an neighbour
        embedding[i, :] -= curr_grad*learning_rate
        embedding[j, :] += curr_grad*learning_rate
        
        # Sample N random points and use them as "negative sample"
        # Negative sample is the sample that is not connected to the current sample
        for _ in range(n_negative):
            
            # Get randomly sample index
            neg_j = np.random.randint(embedding.shape[0])
            
            # Get embedding of negative sample
            p_neg = embedding[neg_j, : ]
            
            # Find euclidean distance between current sample and 
            # negative sample
            dist_squared = np.sum(np.square(p1-p_neg))
            
            # Calculate gradient (formula is shorter because we assume weights[i]==0)
            # for thus case
            if dist_squared > 0.:
                curr_grad = -2.*b / ((0.001 + dist_squared)*(1+a*dist_squared**b))
            else:
                curr_grad = 0.

            curr_grad = curr_grad*(p1-p_neg)
            
            # Clip the gradient
            curr_grad = np.clip(curr_grad, -4., 4.)
            
            # Apply gradient to the both, sample and the neighbour
            embedding[i, :] -=  curr_grad*learning_rate
            embedding[neg_j, :] +=  curr_grad*learning_rate

# Optimize the function            
optimize_fn = numba.njit(
    euclidean_optim_one_epoch, fastmath=True, parallel=True
)

In [ ]:
# Initialize embedding with random values
embedding = np.random.uniform(
    low=-3.0, high=3.0, size=(sim_mtx.shape[0], N_COMPONENTS)
)

In [ ]:
# Optimize in a loop
for epoch_i in range(N_EPOCHS):
    optimize_fn(
        sim_mtx_row_ids=sim_mtx_row_ids, 
        sim_mtx_col_ids=sim_mtx_col_ids, 
        a=a, 
        b=b, 
        embedding=embedding, 
        weights=sim_mtx_vals,
        learning_rate=LEARNING_RATE,
        n_negative=N_NEGATIVE
    )
    
    if epoch_i%100 == 0:
        print(epoch_i)
        test_df = pd.DataFrame(embedding, columns=['UMAP1', 'UMAP2'])
        test_df['label'] = mnist_label.values
        sns.scatterplot(test_df,x='UMAP1', y='UMAP2', hue='label', alpha=0.8, s=8)
        plt.title('Epoch ' + str(epoch_i))
        plt.show()